## Chapter 18. asyncio를 이용한 동시성
Concurrency is Not Paralleism!. 일반적인 운용 환경에서도 100개 이상의 프로세스를 일상적으로 실행한다. 진짜 병렬로 처리하려면 CPU 코어가 여러 개 있어야 한다. 따라서 실제로는 대부분의 처리가 동시에 수행되지만 병렬로 수행되지는 않는다.

이 장에서는 이벤트 루프에 의해 운용되는 코루틴을 이용해서 동시성을 구현하는 asyncio 패키지에 대해 설명한다.

### 18.1 스레드와 코루틴 비교


In [2]:
""" [예제 18-1] spinner_thread.py : 스레드로 텍스트 스피터 애니메이트하기 """
import threading
import itertools
import time
import sys

class Signal:
    go = True
    
def spin(msg, signal):
    write, flush = sys.stdout.write, sys.stdout.flush
    for char in itertools.cycle('|/-\\'):
        status = char + ' ' + msg
        write(status)
        flush()
        write('\x08' * len(status)) # 문자열 길이만큼 백스페이스를 반복해서 커서를 앞으로 이동시킴
        time.sleep(.1)
        if not signal.go:
            break
    write(' ' * len(status) + '\x08' * len(status)) # 스페이스로 기존 문자열을 지우고 백스페이스로 원래 위치로 돌아감
    
def slow_function():
    # 입출력을 위해 장시간 기다리는 것 처럼 보임
    time.sleep(3)
    return 42

def supervisor():
    signal = Signal()
    spinner = threading.Thread(target=spin,
                               args=('thinking!', signal))
    print('spinner object:', spinner)
    spinner.start()
    result = slow_function()
    signal.go = False
    spinner.join()
    return result

def main():
    result = supervisor()
    print('Answer:', result)
    
if __name__ == '__main__':
    main()

spinner object: <Thread(Thread-5, initial)>
| thinking/ thinking- thinking\ thinking| thinking/ thinking- thinking\ thinking| thinking/ thinking- thinking\ thinking| thinking/ thinking- thinking\ thinking| thinking/ thinking- thinking\ thinking| thinking/ thinking- thinking\ thinking| thinking/ thinking- thinking\ thinking| thinking/ thinking          Answer: 42
